In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Load data from Excel (replace '../data.xlsx' with the correct path if needed)
df = pd.read_excel('../data.xlsx', sheet_name='Rekapan')

In [ ]:

# Filter relevant columns
data_filtered = df[['Tanggal', 'SO', 'TERKIRIM', 'Harga Komoditas Bijih Besi', 'Indeks Produksi Dalam Negeri', 'Data Inflasi', 'Kurs']]

# Rename columns for easier handling
data_filtered.columns = ['Tanggal', 'SO', 'Terkirim', 'Harga Komoditas', 'Indeks Produksi', 'Data Inflasi', 'Kurs']

# Convert 'Tanggal' column to datetime
data_filtered['Tanggal'] = pd.to_datetime(data_filtered['Tanggal'])

# Convert 'Indeks Produksi' to numeric (coercing errors)
data_filtered['Indeks Produksi'] = pd.to_numeric(data_filtered['Indeks Produksi'], errors='coerce')

# Create a 'Year' column
data_filtered['Year'] = data_filtered['Tanggal'].dt.year

# Annual Summary based on 'Year'
yearly_summary = data_filtered.groupby('Year').agg({
    'SO': 'sum',
    # Uncomment the following if you need these additional columns
    # 'Terkirim': 'sum',
    # 'Harga Komoditas': 'mean',
    # 'Indeks Produksi': 'mean',
    # 'Data Inflasi': 'mean',
    # 'Kurs': 'mean'
}).reset_index()

# Display annual summary
print(yearly_summary)

In [ ]:
# Set the alpha value for SES (Smoothing Factor)
alpha = 0.1

# Initialize columns for SES forecast and error metrics
yearly_summary['SES Forecast'] = np.nan
yearly_summary['Error'] = np.nan
yearly_summary['MAD'] = np.nan
yearly_summary['MSE'] = np.nan
yearly_summary['MAPE'] = np.nan

# Set initial SES forecast as the first 'SO' value
yearly_summary.loc[0, 'SES Forecast'] = yearly_summary.loc[0, 'SO']

# Apply SES forecasting formula and calculate error metrics
for i in range(1, len(yearly_summary)):
    # SES forecast calculation
    yearly_summary.loc[i, 'SES Forecast'] = (alpha * yearly_summary.loc[i-1, 'SO']) + ((1 - alpha) * yearly_summary.loc[i-1, 'SES Forecast'])
    
    # Error calculation
    yearly_summary.loc[i, 'Error'] = yearly_summary.loc[i, 'SO'] - yearly_summary.loc[i, 'SES Forecast']
    
    # Mean Absolute Deviation (MAD)
    yearly_summary.loc[i, 'MAD'] = abs(yearly_summary.loc[i, 'Error'])
    
    # Mean Squared Error (MSE)
    yearly_summary.loc[i, 'MSE'] = yearly_summary.loc[i, 'Error'] ** 2
    
    # Mean Absolute Percentage Error (MAPE)
    if yearly_summary.loc[i, 'SO'] != 0:
        yearly_summary.loc[i, 'MAPE'] = (abs(yearly_summary.loc[i, 'Error']) / yearly_summary.loc[i, 'SO']) * 100
    else:
        yearly_summary.loc[i, 'MAPE'] = np.nan

# Display SES forecast results and error metrics
print(yearly_summary)

In [ ]:
# Forecast future data with dummy values for SO for upcoming years (for example, next 5 years)
years_to_forecast = 6
last_year = yearly_summary['Year'].max()

# Create dummy data for future years
dummy_years = pd.DataFrame({
    'Year': range(last_year + 1, last_year + 1 + years_to_forecast),
    'SO': np.zeros(years_to_forecast)
})

# Combine the original and dummy data
extended_data = pd.concat([yearly_summary[['Year', 'SO']], dummy_years], ignore_index=True)

# Initialize SES forecast and error metrics for the extended data
extended_data['SES Forecast'] = np.nan
extended_data['Error'] = np.nan
extended_data['MAD'] = np.nan
extended_data['MSE'] = np.nan
extended_data['MAPE'] = np.nan

# Set initial forecast for extended data to the last known SES forecast
extended_data.loc[0, 'SES Forecast'] = extended_data.loc[0, 'SO']

# Apply SES formula for the extended dataset
for i in range(1, len(extended_data)):
    extended_data.loc[i, 'SES Forecast'] = (alpha * extended_data.loc[i-1, 'SO']) + ((1 - alpha) * extended_data.loc[i-1, 'SES Forecast'])
    
    # Calculate the error metrics for future dummy data
    extended_data.loc[i, 'Error'] = extended_data.loc[i, 'SO'] - extended_data.loc[i, 'SES Forecast']
    extended_data.loc[i, 'MAD'] = abs(extended_data.loc[i, 'Error'])
    extended_data.loc[i, 'MSE'] = extended_data.loc[i, 'Error'] ** 2
    if extended_data.loc[i, 'SO'] != 0:
        extended_data.loc[i, 'MAPE'] = (abs(extended_data.loc[i, 'Error']) / extended_data.loc[i, 'SO']) * 100
    else:
        extended_data.loc[i, 'MAPE'] = np.nan

# Display the extended data with forecast results
print(extended_data)

In [ ]:
# Plot: Actual SO vs SES Forecast
plt.figure(figsize=(14, 6))
plt.plot(extended_data['Year'], extended_data['SO'], label='Actual SO', marker='o')
plt.plot(extended_data['Year'], extended_data['SES Forecast'], label='SES Forecast', marker='x')
plt.xlabel('Year')
plt.ylabel('SO')
plt.xticks(rotation=45)
plt.title('Actual SO vs SES Forecast')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Plot: Error Evaluation
plt.figure(figsize=(14, 8))
plt.plot(extended_data['Year'].astype(str), extended_data['Error'], label='Error', marker='o', color='red')
plt.plot(extended_data['Year'].astype(str), extended_data['MAD'], label='MAD', marker='x', color='blue')
plt.plot(extended_data['Year'].astype(str), extended_data['MSE'], label='MSE', marker='s', color='green')
plt.plot(extended_data['Year'].astype(str), extended_data['MAPE'], label='MAPE', marker='^', color='purple')

plt.xlabel('Month-Year')
plt.ylabel('Error Metrics')
plt.xticks(rotation=45)
plt.title('Error and Evaluation Metrics (Error, MAD, MSE, MAPE)')
plt.legend()
plt.tight_layout()
plt.show()